In [ ]:
# Assignment 05
# ANN ~  Artificial Neural Networks 
# Date : 12-10-2025

# Done By : JOSHUA DAVID 

In [62]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt

In [63]:
df = pd.read_csv('mobile_price_classification.csv')

In [64]:
print("First 5 rows of the dataset:")
print(df.head())
print("\nDataset Information:")
df.info()

First 5 rows of the dataset:
   battery_power  bluetooth  clock_speed  dual_sim  front_cam  4G  int_memory  \
0            842          0          2.2         0          1   0           7   
1           1021          1          0.5         1          0   1          53   
2            563          1          0.5         1          2   1          41   
3            615          1          2.5         0          0   0          10   
4           1821          1          1.2         0         13   1          44   

   m_dep  mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  \
0    0.6        188        2  ...         20       756  2549     9     7   
1    0.7        136        3  ...        905      1988  2631    17     3   
2    0.9        145        5  ...       1263      1716  2603    11     2   
3    0.8        131        6  ...       1216      1786  2769    16     8   
4    0.6        141        2  ...       1208      1212  1411     8     2   

   talk_time  three_g  touc

In [65]:
n_features = X_train.shape[1]
n_classes = len(np.unique(y_train))


In [66]:
baseline_model = keras.Sequential([
    keras.layers.Input(shape=(n_features,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(n_classes, activation='softmax')
])


In [68]:
baseline_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

baseline_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 32)             │           672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 4)              │            68 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,268 (4.95 KB)

 Trainable params: 1,268 (4.95 KB)

 Non-trainable params: 0 (0.00 B)

In [70]:
history = baseline_model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2, 
    verbose=1
)

Epoch 1/100


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9875 - loss: 0.1016 - val_accuracy: 0.9281 - val_loss: 0.1997
Epoch 2/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9859 - loss: 0.0977 - val_accuracy: 0.9344 - val_loss: 0.1961
Epoch 3/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9891 - loss: 0.0920 - val_accuracy: 0.9281 - val_loss: 0.1937
Epoch 4/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9867 - loss: 0.0870 - val_accuracy: 0.9219 - val_loss: 0.1990
Epoch 5/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9891 - loss: 0.0845 - val_accuracy: 0.9219 - val_loss: 0.2023
Epoch 6/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9906 - loss: 0.0791 - val_accuracy: 0.9375 - val_loss: 0.1862
Epoch 7/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9914 - loss: 0.0747 - val_accuracy: 0.9281 - val_loss: 0.1890
Epoch 8/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9922 - loss: 0.0713 - val_accuracy: 0.9281 - val_loss: 0.1

In [72]:
loss, accuracy = baseline_model.evaluate(X_test, y_test)
print(f"\nBaseline Model Test Accuracy: {accuracy*100:.2f}%")

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9250 - loss: 0.2351  

Baseline Model Test Accuracy: 92.50%


In [73]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(n_features,)))

    # Tune the number of hidden layers and neurons in each
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(keras.layers.Dense(
            units=hp.Int(f'units_{i}', min_value=32, max_value=256, step=32),
            activation='relu'
        ))

    model.add(keras.layers.Dense(n_classes, activation='softmax'))

    # Tune the learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [74]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='keras_tuner_dir',
    project_name='mobile_price'
)

In [75]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [76]:

print("\nStarting hyperparameter search...")
tuner.search(
    X_train, y_train,
    epochs=50,
    validation_split=0.2,
    callbacks=[stop_early],
    verbose=1 )

Trial 30 Complete [00h 00m 05s]
val_accuracy: 0.512499988079071

Best val_accuracy So Far: 0.9312499761581421
Total elapsed time: 00h 01m 30s


In [77]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


In [78]:
print(f"""
The hyperparameter search is complete.
- Optimal number of layers: {best_hps.get('num_layers')}
- Optimal learning rate: {best_hps.get('learning_rate')}
""")


The hyperparameter search is complete.
- Optimal number of layers: 1
- Optimal learning rate: 0.01



In [79]:
final_model = tuner.hypermodel.build(best_hps)
final_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 256)            │         5,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,404 (25.02 KB)

 Trainable params: 6,404 (25.02 KB)

 Non-trainable params: 0 (0.00 B)

In [80]:
print("\nTraining the final, tuned model...")
history_final = final_model.fit(
    X_train, y_train,
    epochs=100, # Train for more epochs with early stopping
    validation_split=0.2,
    callbacks=[stop_early],
    verbose=0
)


Training the final, tuned model...


In [81]:
final_loss, final_accuracy = final_model.evaluate(X_test, y_test)
print(f"\nFinal Tuned Model Test Accuracy: {final_accuracy*100:.2f}%")

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9150 - loss: 0.2134  

Final Tuned Model Test Accuracy: 91.50%


In [82]:
final_model.save('mobile_price_model.h5')

In [83]:
import pickle
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


In [84]:
# ---- TESTINGGGG 

In [ ]:
import numpy as np
import tensorflow as tf
import pickle

loaded_model = tf.keras.models.load_model('mobile_price_model.h5')
with open('scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

# battery_power,bluetooth,clock_speed,dual_sim,front_cam,4G,int_memory,m_dep,mobile_wt,n_cores,primary_camera,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range

new_phone_features = np.array([[
    1800, 1, 2.8, 1, 16, 1, 64, 0.5, 150, 8, 20, 1400, 1800, 4096, 18, 9, 20, 1, 1, 1
]])

scaled_features = loaded_scaler.transform(new_phone_features)

prediction = loaded_model.predict(scaled_features)
predicted_class = np.argmax(prediction[0])

print(f"The predicted price range for the new mobile is: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
The predicted price range for the new mobile is: 3


/home/jodave/Desktop/ICT Acadymy/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [87]:
price_labels = {
    0: 'Low Cost',
    1: 'Medium Cost',
    2: 'High Cost',
    3: 'Very High Cost'
}


In [88]:
loaded_model = tf.keras.models.load_model('mobile_price_model.h5')
with open('scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)
print("Model and scaler loaded successfully.\n")

Model and scaler loaded successfully.



In [89]:
# battery_power,bluetooth,clock_speed,dual_sim,front_cam,4G,int_memory,m_dep,mobile_wt,n_cores,primary_camera,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range


In [90]:
test_phones = np.array([
    # Test1 A budget-friendly phone
    [1000, 0, 1.2, 1, 2, 0, 16, 0.8, 150, 2, 5, 240, 320, 768, 10, 5, 12, 1, 1, 1],

    # Test 2 A solid mid-range phone
    [1400, 1, 2.0, 1, 8, 1, 64, 0.5, 140, 4, 13, 720, 1280, 2048, 14, 7, 15, 1, 1, 1],

    # Test 3 A high-end phone
    [1800, 1, 2.8, 1, 16, 1, 128, 0.7, 160, 8, 24, 1080, 1920, 4096, 16, 8, 18, 1, 1, 1],

    # Test 4 An ultra-premium flagship phone
    [1950, 1, 3.0, 1, 20, 1, 256, 0.6, 170, 8, 48, 1440, 2560, 8192, 18, 9, 20, 1, 1, 1]
])


In [91]:
scaled_test_phones = loaded_scaler.transform(test_phones)

/home/jodave/Desktop/ICT Acadymy/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [92]:
predictions = loaded_model.predict(scaled_test_phones)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


In [94]:
for i, prediction in enumerate(predictions):
    predicted_class = np.argmax(prediction)
    confidence = np.max(prediction)
    label = price_labels[predicted_class]

    print(f"\n--- Test Phone {i+1} ---")
    print(f"Predicted Price Range: '{label}' (Class {predicted_class})")
    print(f"Model Confidence: {confidence*100:.2f}%")



--- Test Phone 1 ---
Predicted Price Range: 'Low Cost' (Class 0)
Model Confidence: 100.00%

--- Test Phone 2 ---
Predicted Price Range: 'High Cost' (Class 2)
Model Confidence: 74.82%

--- Test Phone 3 ---
Predicted Price Range: 'Very High Cost' (Class 3)
Model Confidence: 100.00%

--- Test Phone 4 ---
Predicted Price Range: 'Very High Cost' (Class 3)
Model Confidence: 100.00%
